# 术语名词解释

## DTS(Decoding Time Stamp) 和 PTS(Presentation Time Stamp)

H264里有两种时间戳：DTS（Decoding Time Stamp）和PTS（Presentation Time Stamp）。

DTS 指的是解码时间 
PTS 指的是显示时间 

FFmpeg 中用 AVPacket 结构体来描述解码前或编码后的压缩包，
用AVFrame结构体来描述解码后或编码前的信号帧。

对于视频来说，AVFrame就是视频的一帧图像。
- 这帧图像什么时候显示给用户，就取决于它的PTS。
- DTS是AVPacket里的一个成员，表示这个压缩包应该什么时候被解码。

>需要注意的是, 如果视频里各帧的编码是按输入顺序（也就是显示顺序）依次进行的，
>那么解码和显示时间应该是一致的。

>可事实上，在大多数编解码标准（如H.264或HEVC）中，编码顺序和输入顺序并不一致。 
>于是才会需要PTS和DTS这两种不同的时间戳。

## GOP(Group of Picture)
GOP (Group of picture)，从字面意思来看，GOP 就是指一组 Slice，但是在 GOP 的具体定义上，有好几个版本

1. 出处不详，GOP 是从一个 I Slice 开始，直到碰到下一个 I Slice 结束，这个区间范围内的所有 Slice。这个版本显然是传播最广的，也是最容易理解的。
2. 出自 x264，x264 对 GOP 进行了新的发明创造，创造除了 Open-GOP 和 Close-GOP 两种概念。

- Open-GOP， Open-GOP 由一个 I Slice 开始，Open 是指这个 GOP 是开放的，在这个 GOP 中的 P Slice 和 B Slice 可以参考这个 GOP 开始的 I Slice 之前的 Slice。

![图 1](../asset/b1b56968fd4f3515c06d299ee692e37037ebce63eb83f5b3c5b696b8b95aa7bf.png)  


- Close-GOP，Close-GOP 由一个 I Slice 开始，Close 是指这个 GOP 是封闭的，在这个 GOP 中的 P Slice 和 B Slice 不可以参考这个 GOP 开始的 I Slice 之前的 Slice。

![图 2](../asset/f820a91f66f6e2d1ddaad2b32eed39fb976f33db59a959c2acd29db0307f1daf.png)  

其实过度纠结 GOP 的定义并没有太多实际的意义，GOP 的概念其实也不是 H.264 标准的概念，而是在生产中，大家总结出来的一个方便理解的概念，对其进行过度解读，反而会造成不必要的误解。

## 与多媒体数据相关的几个概念

- track 表示一些sample的集合，对于媒体数据来说，track表示一个视频或音频序列。注意mp4中的track本质是对实际的media data位置&大小的描述，是抽象的不是具体的。
- hint track 这个特殊的track并不包含媒体数据，而是包含了一些将其他数据track打包成流媒体的指示信息。
- sample 对于非hint track来说，video sample即为一帧或一组连续视频帧，audio sample即为一段连续的压缩音频，它们统称sample。
- sample table 描述sampe的时序 和物理布局的表。
- chunk 一个track的几个sample组成的单元。
- stream 有audio stream和video stream，一般指具体的音频/视频流数据，可以看成是MP4容器中的track+mdat之和

>[注1]：audio的一帧可分解成多个sample，所以采用sample为最小单位。对于hint track，sample定义一个或多个流媒体包的格式。  
>[注2]：每个track还会含一个或者多个sample descriptions。track里面的每个sample通过引用关联到一个sample description。这个sample descriptions定义了怎样解码这个sample & 使用什么解压缩算法

# MP4文件格式概览

MP4 格式在解码播放中的位置:
![图 1](../asset/405b99d58439f65ca31de3b5e7df9156ef9964cc397e1a817b38000d1f8d61ee.png)  

MP4文件由多个box组成，每个box存储不同的信息，且box之间是树状结构，如下图所示。

![图 2](../asset/fa715aef84b6d082e1d20e6fb25febbc497fbd4150989dcd6036187d18815fe2.png)  


box类型有很多，下面是3个比较重要的顶层box：

- ftyp: 类型box：一个MP4文件的开头会有且只有一个 “ftyp” 类型的box，作为MP4格式的标志并包含关于文件的一些信息：版本、兼容协议等；
- moov: 类型box：之后会有且只有一个“moov”类型的box（Movie Box），它是一种container box，子box包含了媒体的 metadata 信息；
- mdat: 类型box：MP4文件的媒体数据包含在“mdat”类型的box（Media Data Box）中，该类型的box也是container box，可以有多个，也可以没有（当媒体数据全部引用其他文件时），媒体数据的结构由metadata进行描述（位于moov--->udta--->meta）。

![图 1](../asset/ae490519726a0c4d0d415d2fa513ffe6c5f82b439b68fca7e9808445f03fa51d.png)  

下面是一个典型的MP4文件结构:

![图 3](../asset/9c0e13f30a4e0ccacab680a85eba8d4c1ca9c903258b90f5f32592a35bcb3ce6.png)  


下面是一些常见的Box的简介:

![图 4](../asset/56baa44dd65fd084b02f3e0a5f4a4b74ce859d947aa70f8c73f6837277e2e2d7.png)  

In [33]:
# f = open("./data/data1.mp4", "rb")
f = open(r"M:\RepairResource\Video\#VI_0161_197.38.116.158.MP4", "rb")
data = f.read()
f.close()

kBoxNames = ['ftyp', 'moov', 'mdat', 'mvhd', 'trak', 'tkhd', 'mdia', 'mdhd', 'hdlr', 'minf', 'stbl', 'stsd', 'stts', 'ctts', 'stsc', 'stsz', 'stz2', 'stss', 'stco', 'co64']
def get_all_box_offset(data):
    box_offsets = {}
    for box in kBoxNames:
        box = box.encode()
        offset = data.find(box)
        if offset == -1:
            continue
        box_offsets[box.decode()] = [offset]
        while True:
            offset = data.find(box, offset + 1)
            if offset == -1:
                break
            box_offsets[box.decode()].append(offset)
    return box_offsets

box_offsets = get_all_box_offset(data)
for box, offsets in box_offsets.items():
    print(f'{box}: {[hex(offset) for offset in offsets]}')

mdat: ['0x178004']


# MP4 Box简介
1个box由两部分组成：box header、box body。

- box header：box的元数据，比如box type、box size。
- box body：box的数据部分，实际存储的内容跟box类型有关，比如mdat中body部分存储的媒体数据。

box header中，只有type、size是必选字段。
当size==0时，存在largesize字段。
在部分box中，还存在version、flags字段，这样的box叫做Full Box。
当box body中嵌套其他box时，这样的box叫做container box。

![图 2](../asset/83a19fed9ecd84867a21e063d62e6c04bb0d5d77ec63d4adbcdfb4055870f884.png)  

## Box Header
字段定义如下：

- type：box类型，包括 “预定义类型”、“自定义扩展类型”，占4个字节；
    - 预定义类型：比如ftyp、moov、mdat等预定义好的类型；
    - 自定义扩展类型：如果type==uuid，则表示是自定义扩展类型。size（或largesize）随后的16字节，为自定义类型的值（extended_type）
- size：包含box header在内的整个box的大小，单位是字节。当size为0或1时，需要特殊处理：
    - size等于0：box的大小由后续的largesize确定（一般只有装载媒体数据的mdat box会用到largesize）；
    - size等于1：当前box为文件的最后一个box，通常包含在mdat box中；
- largesize：box的大小，占8个字节；
- extended_type：自定义扩展类型，占16个字节；

In [5]:
from construct import *

class Box:
    def __init__(self, name=''):
        if not hasattr(self, 'body_fmt'):
            self.body_fmt = IfThenElse(
                this.size == 1,
                GreedyBytes,
                Bytes(
                    this.size - 8 if this.large_size == None else this.large_size - 16
                ),
            )

        self.struct = None
        self.size = None
        self.body = None
        self.box_name = name

        self.fmt = Struct(
            "size" / Int32ub,
            "type" / PaddedString(4, "ascii"),
            "large_size" / Optional(If(this.size == 0, Int64ub)),
            "extended_type" / Optional(If(this.type == "uuid", Bytes(16))),
            "body" / self.body_fmt,
        )

    def __str__(self):
        ret = f"box_name: {self.box_name}\nNormal Box " + self.struct.__str__()
        return ret

    def init(self, data):
        try:
            self.struct = self.fmt.parse(data)
            self.size = (
                self.struct.size
                if self.struct.large_size == None
                else self.struct.large_size
            )
            if self.box_name == '':
                self.box_name = self.struct.type
            self.body = self.struct.body
        except Exception as e:
            print(e)

    @staticmethod
    def getBoxList(data):
        cur_offset = 0
        boxes = []
        data_len = len(data)

        while cur_offset < data_len:
            tmp = Box(data[cur_offset:])

            # print(f"Current pos is {cur_offset}/{data_len}")

            box = None
            if tmp.box_name == "avc1":
                box = VisualSampleEntry(data[cur_offset:])
            else:
                box = tmp

            boxes.append(box)
            cur_offset += box.size

        return boxes

box = Box()
box.init(data)
print(box)


'ascii' codec can't decode byte 0xf7 in position 3: ordinal not in range(128)
box_name: 
Normal Box None


## FullBox
在Box的基础上，扩展出了FullBox类型。相比Box，FullBox 多了 version、flags 字段。

- version：当前box的版本，为扩展做准备，占1个字节；
- flags：标志位，占24位，含义由具体的box自己定义；

FullBox主要在moov中的box用到，比如 moov.mvhd

In [6]:
class FullBox(Box):
    def __init__(self, name=''):
        if not hasattr(self, 'body_fmt'):
            self.body_fmt = IfThenElse(
                this.size == 1,
                GreedyBytes,
                Bytes(
                    this.size - 12 if this.large_size == None else this.large_size - 20
                ),
            )

        super().__init__(name)

        self.version = None
        self.flags = None

        self.fmt = Struct(
            "size" / Int32ub,
            "type" / PaddedString(4, "ascii"),
            "large_size" / Optional(If(this.size == 0, Int64ub)),
            "extended_type" / Optional(If(this.type == "uuid", Bytes(16))),
            "version" / Int8ub,
            "flags" / Bytes(3),
            "body" / self.body_fmt,
        )

    def init(self, data):
        super().init(data)
        self.version = self.struct.version
        self.flags = self.struct.flags

    def __str__(self):
        ret = f"box_name: {self.box_name}\nFull Box " + self.struct.__str__()
        return ret


# ftyp(File Type Box)
ftyp用来指出当前文件遵循的规范

## 什么是isom
isom（ISO Base Media file）是在 MPEG-4 Part 12 中定义的一种基础文件格式，MP4、3gp、QT 等常见的封装格式，都是基于这种基础文件格式衍生的。

MP4 文件可能遵循的规范有mp41、mp42，而mp41、mp42又是基于isom衍生出来的。

>3gp(3GPP)：一种容器格式，主要用于3G手机上；  
>QT：QuickTime的缩写，.qt 文件代表苹果QuickTime媒体文件；

## ftyp 定义
其头部信息和 Box 一致, 主要是 body 部分:
- major_brand(4 字节字符串): 比如常见的 isom、mp41、mp42、avc1、qt等。它表示“最好”基于哪种格式来解析当前的文件。
- minor_version(4 字节无符号整数): 提供 major_brand 的说明信息，比如版本号，不得用来判断媒体文件是否符合某个标准/规范
- compatible_brands(4 字节字符串列表): 文件兼容的brand列表。比如 mp41 的兼容 brand 为 isom。通过兼容列表里的 brand 规范，可以将文件 部分（或全部）解码出来；

>在实际使用中，不能把 isom 做为 major_brand，而是需要使用具体的brand（比如mp41），  
>因此，对于 isom，没有定义具体的文件扩展名、mime type。

下面是常见的几种brand，以及对应的文件扩展名、mime type，更多brand可以参考 [这里](http://fileformats.archiveteam.org/wiki/Boxes/atoms_format#Brands) 

![图 6](../asset/b3cfd18009831c5b910c8fa048d1ed88785f6f6f87e5e3f9a5f8c34872422979.png)  




In [13]:
class FileTypeBox(Box):
    def __init__(self):
        self.body_fmt = Struct(
            "major_brand" / PaddedString(4, "ascii"),
            "minor_version" / Int32ub,
            "compatible_brands" / GreedyRange(PaddedString(4, "ascii")),
        )

        super().__init__('ftyp')


ftyp_offset = data.find(b"ftyp") - 4
if ftyp_offset >= 0:
    ftyp = FileTypeBox()
    ftyp.init(data)
    print(ftyp)


# moov(Movie Box)

Movie Box，存储 mp4 的 metadata，一般位于mp4文件的开头。

moov中，最重要的两个box是 mvhd 和 trak：

- mvhd：Movie Header Box，mp4文件的整体信息，比如创建时间、文件时长等；
- trak：Track Box，一个mp4可以包含一个或多个轨道（比如视频轨道、音频轨道），轨道相关的信息就在trak里。trak是container box，至少包含两个box，tkhd、mdia；

>mvhd针对整个影片，tkhd针对单个track，mdhd针对媒体，vmhd针对视频，smhd针对音频，  
>可以认为是从 宽泛 > 具体，前者一般是从后者推导出来的。

## mvhd(Movie Header Box)

MP4文件的整体信息，跟具体的视频流、音频流无关，比如创建时间、文件时长等。

定义如下:
```java
aligned(8) class MovieHeaderBox extends FullBox(‘mvhd’, version, 0) { 
    if (version==1) {
      unsigned int(64)  creation_time;
      unsigned int(64)  modification_time;
      unsigned int(32)  timescale;
      unsigned int(64)  duration;
    } else { // version==0
      unsigned int(32)  creation_time;
      unsigned int(32)  modification_time;
      unsigned int(32)  timescale;
      unsigned int(32)  duration;
    }
    template int(32) rate = 0x00010000; // typically 1.0
    template int(16) volume = 0x0100; // typically, full volume 
    const bit(16) reserved = 0;
    const unsigned int(32)[2] reserved = 0;
    template int(32)[9] matrix =
        { 0x00010000,0,0,0,0x00010000,0,0,0,0x40000000 };
        // Unity matrix
    bit(32)[6]  pre_defined = 0;
    unsigned int(32)  next_track_ID;
}
```
含义如下:

- creation_time：文件创建时间；
- modification_time：文件修改时间；
- timescale：一秒包含的时间单位（整数）。举个例子，如果timescale等于1000，那么，一秒包含1000个时间单位（后面track等的时间，都要用这个来换算，比如track的duration为10,000，那么，track的实际时长为10,000/1000=10s）；
- duration：影片时长（整数），根据文件中的track的信息推导出来，等于时间最长的track的duration；
- rate：推荐的播放速率，32位整数，高16位、低16位分别代表整数部分、小数部分（[16.16]），举例 0x0001 0000 代表1.0，正常播放速度；
- volume：播放音量，16位整数，高8位、低8位分别代表整数部分、小数部分（[8.8]），举例 0x01 00 表示 1.0，即最大音量；
- matrix：视频的转换矩阵，一般可以忽略不计；
- next_track_ID：32位整数，非0，一般可以忽略不计。当要添加一个新的track到这个影片时，可以使用的track id，必须比当前已经使用的track id要大。也就是说，添加新的track时，需要遍历所有track，确认可用的track id；

In [14]:
class MovieHeaderBox(FullBox):
    def __init__(self):
        self.body_fmt = Struct(
            "creation_time" / IfThenElse(this._root.version == 1, Int64ub, Int32ub),
            "modification_time" / IfThenElse(this._root.version == 1, Int64ub, Int32ub),
            "timescale" / Int32ub,
            "duration" / IfThenElse(this._root.version == 1, Int64ub, Int32ub),
            "rate" / Bytes(4),
            "volume" / Bytes(2),
            "reserved1" / Bytes(2),
            "reserved2" / Bytes(8),
            "matrix" / Bytes(4)[9],
            "pre_defined" / Bytes(4)[6],
            "next_track_ID" / Int32ub,
        )

        super().__init__('mvhd')

    def init(self, data):
        super().init(data)
        self.body.rate = self.convertRate(self.body.rate)
        self.body.volume = self.convertVolume(self.body.volume)

    def convertRate(self, rate_byte):
        high = int.from_bytes(rate_byte[:2], byteorder="big")
        low = int.from_bytes(rate_byte[2:], byteorder="big")
        return str(high) + "." + str(low)

    def convertVolume(self, volume_byte):
        high = int.from_bytes(volume_byte[:1], byteorder="big")
        low = int.from_bytes(volume_byte[1:], byteorder="big")
        return str(high) + "." + str(low)


mvhd_offset = data.find(b"mvhd") - 4
if mvhd_offset >= 0:
    mvhd = MovieHeaderBox()
    mvhd.init(data[mvhd_offset:])
    # print(mvhd.body)
    print(mvhd)


## trak(Track Box)

Track Box 是一个 Container Box, 里面包含至少两个 Box, tkhd(Track Header Box), media
一个mp4可以包含一个或多个轨道（比如视频轨道、音频轨道），轨道相关的信息就在 trak 里。

定义如下: 
```java
aligned(8) class TrackBox extends Box(‘trak’) { 
} 
```

### thkd(Track Header Box)

定义如下:

```java
aligned(8) class TrackHeaderBox 
  extends FullBox(‘tkhd’, version, flags){ 
	if (version==1) {
	      unsigned int(64)  creation_time;
	      unsigned int(64)  modification_time;
	      unsigned int(32)  track_ID;
	      const unsigned int(32)  reserved = 0;
	      unsigned int(64)  duration;
	   } else { // version==0
	      unsigned int(32)  creation_time;
	      unsigned int(32)  modification_time;
	      unsigned int(32)  track_ID;
	      const unsigned int(32)  reserved = 0;
	      unsigned int(32)  duration;
	}
	const unsigned int(32)[2] reserved = 0;
	template int(16) layer = 0;
	template int(16) alternate_group = 0;
	template int(16) volume = {if track_is_audio 0x0100 else 0}; 
    const unsigned int(16) reserved = 0;
	template int(32)[9] matrix= { 0x00010000,0,0,0,0x00010000,0,0,0,0x40000000 }; // unity matrix
	unsigned int(32) width;
	unsigned int(32) height;
}
```

单个 track 的 metadata，包含如下字段：

- version：tkhd box的版本；
- flags：按位或操作获得，默认值是7（0x000001 | 0x000002 | 0x000004），表示这个track是启用的、用于播放的 且 用于预览的。
    - Track_enabled：值为0x000001，表示这个track是启用的，当值为0x000000，表示这个track没有启用；
    - Track_in_movie：值为0x000002，表示当前track在播放时会用到；
    - Track_in_preview：值为0x000004，表示当前track用于预览模式；
- creation_time：当前track的创建时间；
- modification_time：当前track的最近修改时间；
- track_ID：当前track的唯一标识，不能为0，不能重复；
- duration：当前track的完整时长（需要除以timescale得到具体秒数）；
- layer：视频轨道的叠加顺序，数字越小越靠近观看者，比如1比2靠上，0比1靠上；
- alternate_group：当前track的分组ID，alternate_group值相同的track在同一个分组里面。同个分组里的track，同一时间只能有一个track处于播放状态。当alternate_group为0时，表示当前track没有跟其他track处于同个分组。一个分组里面，也可以只有一个track；
- volume：audio track的音量，介于0.0~1.0之间；
- matrix：视频的变换矩阵；
- width、height：视频的宽高；

In [15]:
class TrackBox(Box):
    def __init__(self):
        super().__init__('trak')


class TrackHeaderBox(FullBox):
    def __init__(self):
        self.body_fmt = Struct(
            "creation_time" / IfThenElse(this._root.version == 1, Int64ub, Int32ub),
            "modification_time" / IfThenElse(this._root.version == 1, Int64ub, Int32ub),
            "track_ID" / Int32ub,
            "reserved1" / Int32ub,
            "duration" / IfThenElse(this._root.version == 1, Int64ub, Int32ub),
            "reserved2" / Bytes(8),
            "layer" / Int16sb,
            "alternate_group" / Int16sb,
            "volume" / Bytes(2),
            "reserved3" / Bytes(2),
            "matrix" / Bytes(4)[9],
            "width" / Aligned(4, Int16ub),
            "height" / Aligned(4, Int16ub),
        )

        super().__init__('thkd')

    def init(self, data):
        super().init(data)

        self.body.volume = self.convertVolume(self.body.volume)

    def convertVolume(self, volume_byte):
        high = int.from_bytes(volume_byte[:1], byteorder="big")
        low = int.from_bytes(volume_byte[1:], byteorder="big")
        return str(high) + "." + str(low)


trak_offset = data.find(b"trak") - 4
if trak_offset >= 0:
    trak = TrackBox()
    trak.init(data[trak_offset:])
    print(trak)
    print()
    thkd = TrackHeaderBox()
    thkd.init(trak.body)
    # print(thkd.body)
    print(thkd)


### mdhd(Media Header Box)

主要是用来存放 track 中视频流创建时间，长度, 时间基准等信息。

![图 3](../asset/cd219425a4e6132393c5dc3314b05deaee986280e25264974bb0ac5b0f748cbb.png)  


```java
aligned(8) class MediaHeaderBox extends FullBox(‘mdhd’, version, 0) { 
 if (version==1) { 
  unsigned int(64)  creation_time; 
  unsigned int(64)  modification_time; 
  unsigned int(32)  timescale; 
  unsigned int(64)  duration; 
 } else { // version==0 
  unsigned int(32)  creation_time; 
  unsigned int(32)  modification_time; 
  unsigned int(32)  timescale; 
  unsigned int(32)  duration; 
 } 
 bit(1) pad = 0; 
 unsigned int(5)[3] language; // ISO-639-2/T language code 
 unsigned int(16)  pre_defined = 0; 
}
```

In [16]:
class MediaHeaderBox(FullBox):
    def __init__(self):
        self.body_fmt = Struct(
            "creation_time" / IfThenElse(this._root.version == 1, Int64ub, Int32ub),
            "modification_time" / IfThenElse(this._root.version == 1, Int64ub, Int32ub),
            "timescale" / Int32ub,
            "duration" / IfThenElse(this._root.version == 1, Int64ub, Int32ub),
            "language_bit" / BitStruct(
                "pad" / Bit,
                "language" / BitsInteger(5)[3],
            ),
            "pre_defined" / Int16ub,
        )

        super().__init__("mdhd")


mdhd_offset = data.find(b"mdhd") - 4
if mdhd_offset >= 0:
    mdhd = MediaHeaderBox()
    mdhd.init(data[mdhd_offset:])
    print(mdhd)


#### hdlr(Handler Reference Box)

声明当前track的类型，以及对应的处理器（handler）。

定义如下:
```java
aligned(8) class HandlerBox extends FullBox(‘hdlr’, version = 0, 0) { 
	unsigned int(32) pre_defined = 0;
	unsigned int(32) handler_type;
	const unsigned int(32)[3] reserved = 0;
   	string   name;
}
```

字段含义如下:
- handler_type: 有如下类型
    - vide（0x76 69 64 65），video track；
    - soun（0x73 6f 75 6e），audio track；
    - hint（0x68 69 6e 74），hint track；
- name: 为utf8字符串，对handler进行描述，比如 L-SMASH Video Handler（参考 [这里](http://avisynth.nl/index.php/LSMASHSource)）。

In [17]:
class HandlerBox(FullBox):
    def __init__(self):
        self.body_fmt = Struct(
            "pre_defined" / Int32ub,
            "handler_type" / Bytes(4),
            "reserved" / Bytes(12),
            "name" / CString('utf8')
        )

        super().__init__('hdlr')


hdlr_offset = data.find(b"hdlr") - 4
if hdlr_offset >= 0:
    hdlr = HandlerBox()
    hdlr.init(data[hdlr_offset:])
    print(hdlr)


##### stbl(Sample Table Box)

MP4文件的媒体数据部分在mdat box里，而stbl则包含了这些媒体数据的索引以及时间信息，了解stbl对解码、渲染MP4文件很关键。

在SampleDescriptionBox 中，handler_type 参数 为 track 的类型（soun、vide、hint）
entry_count 变量代表当前box中 smaple description 的条目数。

>[stsc](#stscsample-to-chunk-box) 中，sample_description_index 就是指向这些smaple description的索引。

针对不同的[handler_type](#hdlrhandler-reference-box)，
SampleDescriptionBox 后续应用不同的 SampleEntry 类型，
比如video track为VisualSampleEntry。

VisualSampleEntry包含如下字段：

- data_reference_index：当MP4文件的数据部分，可以被分割成多个片段，每一段对应一个索引，并分别通过URL地址来获取，此时，data_reference_index 指向对应的片段（比较少用到）；
- width、height：视频的宽高，单位是像素；
- horizresolution、vertresolution：水平、垂直方向的分辨率（像素/英寸），16.16定点数，默认是0x00480000（72dpi）；
- frame_count：一个sample中包含多少个frame，对video track来说，默认是1；
- compressorname：仅供参考的名字，通常用于展示，占32个字节，比如 AVC Coding。第一个字节，表示这个名字实际要占用N个字节的长度。第2到第N+1个字节，存储这个名字。第N+2到32个字节为填充字节。compressorname 可以设置为0；
- depth：位图的深度信息，比如 0x0018（24），表示不带alpha通道的图片；

定义如下:
```java
aligned(8) class SampleTableBox extends Box(‘stbl’) { 
}
```

在MP4文件中，媒体数据被分成多个chunk，每个chunk可包含多个sample，而sample则由帧组成（通常1个sample对应1个帧），关系如下：

![图 7](../asset/4d5a52a65e7d6643b54f0509b8ba230aa1c56b6fdb9b0c4cf1fe157be4ab4657.jpg)  


stbl中比较关键的box包含stsd、stco、stsc、stsz、stts、stss、ctts。下面先来个概要的介绍，然后再逐个讲解细节。

###### stco / stsc / stsz / stts / stss / ctts / stsd 概述
下面是这几个box概要的介绍：

- stsd：给出视频、音频的编码、宽高、音量等信息，以及每个sample中包含多少个frame；
- stco：thunk在文件中的偏移；
- stsc：每个thunk中包含几个sample；
- stsz：每个sample的size（单位是字节）；
- stts：每个sample的时长；
- stss：哪些sample是关键帧；
- ctts：帧解码到渲染的时间差值，通常用在B帧的场景；

###### stsd（Sample Description Box）

stsd给出sample的描述信息，这里面包含了在解码阶段需要用到的任意初始化信息，比如 编码 等。对于视频、音频来说，所需要的初始化信息不同，这里以视频为例。

定义如下:
```java
aligned(8) abstract class SampleEntry (unsigned int(32) format) 
 extends Box(format){ 
 const unsigned int(8)[6] reserved = 0; 
 unsigned int(16) data_reference_index; 
} 
 
class HintSampleEntry() extends SampleEntry (protocol) { 
 unsigned int(8) data []; 
} 
 // Visual Sequences 
class VisualSampleEntry(codingname) extends SampleEntry (codingname){ 
 unsigned int(16) pre_defined = 0; 
 const unsigned int(16) reserved = 0; 
 unsigned int(32)[3]  pre_defined = 0; 
 unsigned int(16)  width; 
 unsigned int(16)  height; 
 template unsigned int(32)  horizresolution = 0x00480000; // 72 dpi 
 template unsigned int(32)  vertresolution  = 0x00480000; // 72 dpi 
 const unsigned int(32)  reserved = 0; 
 template unsigned int(16)  frame_count = 1; 
 string[32]  compressorname; 
 template unsigned int(16)  depth = 0x0018; 
 int(16)  pre_defined = -1; 
} 
 
 // Audio Sequences 
 
class AudioSampleEntry(codingname) extends SampleEntry (codingname){ 
 const unsigned int(32)[2] reserved = 0; 
 template unsigned int(16) channelcount = 2; 
 template unsigned int(16) samplesize = 16; 
 unsigned int(16) pre_defined = 0; 
 const unsigned int(16) reserved = 0 ; 
 template unsigned int(32) samplerate = {timescale of media}<<16; 
}

aligned(8) class SampleDescriptionBox (unsigned int(32) handler_type) 
 extends FullBox(’stsd’, 0, 0){ 
 int i ; 
 unsigned int(32) entry_count; 
 for (i = 1 ; i u entry_count ; i++){ 
  switch (handler_type){ 
   case ‘soun’: // for audio tracks 
    AudioSampleEntry(); 
    break; 
   case ‘vide’: // for video tracks  
    VisualSampleEntry(); 
    break; 
   case ‘hint’: // Hint track 
    HintSampleEntry(); 
    break; 
  } 
 } 
}
```

###### avc1, avc2, avcC, m4ds, btrt

这几个box 是 Advanced Video Coding (AVC) file format 编码的定义, 位于 [ISO/IEC 14496-15](https://raw.githubusercontent.com/wiki/ossrs/srs/doc/ISO_IEC_14496-15-AVC-format-2012.pdf), 中


定义如下:

```java
aligned(8) class AVCDecoderConfigurationRecord { 
 unsigned int(8) configurationVersion = 1; 
 unsigned int(8) AVCProfileIndication; 
 unsigned int(8) profile_compatibility; 
 unsigned int(8) AVCLevelIndication;  
 bit(6) reserved = ‘111111’b; 
 unsigned int(2) lengthSizeMinusOne;  
 bit(3) reserved = ‘111’b; 
 unsigned int(5) numOfSequenceParameterSets; 
 for (i=0; i< numOfSequenceParameterSets;  i++) { 
  unsigned int(16) sequenceParameterSetLength ; 
  bit(8*sequenceParameterSetLength) sequenceParameterSetNALUnit; 
 } 
 unsigned int(8) numOfPictureParameterSets; 
 for (i=0; i< numOfPictureParameterSets;  i++) { 
  unsigned int(16) pictureParameterSetLength; 
  bit(8*pictureParameterSetLength) pictureParameterSetNALUnit; 
 } 
 if( profile_idc  ==  100  ||  profile_idc  ==  110  || 
     profile_idc  ==  122  ||  profile_idc  ==  144 ) 
 { 
  bit(6) reserved = ‘111111’b; 
  unsigned int(2) chroma_format; 
  bit(5) reserved = ‘11111’b; 
  unsigned int(3) bit_depth_luma_minus8; 
  bit(5) reserved = ‘11111’b; 
  unsigned int(3) bit_depth_chroma_minus8; 
  unsigned int(8) numOfSequenceParameterSetExt; 
  for (i=0; i< numOfSequenceParameterSetExt; i++) { 
   unsigned int(16) sequenceParameterSetExtLength; 
   bit(8*sequenceParameterSetExtLength) sequenceParameterSetExtNALUnit; 
  } 
 } 
}

// Visual Sequences 
class AVCConfigurationBox extends Box(‘avcC’) { 
 AVCDecoderConfigurationRecord() AVCConfig; 
} 
class MPEG4BitRateBox extends Box(‘btrt’){ 
 unsigned int(32) bufferSizeDB; 
 unsigned int(32) maxBitrate; 
 unsigned int(32) avgBitrate; 
} 
class MPEG4ExtensionDescriptorsBox extends Box(‘m4ds’) { 
 Descriptor Descr[0 .. 255]; 
} 
class AVCSampleEntry() extends VisualSampleEntry (‘avc1’){ 
 AVCConfigurationBox  config; 
 MPEG4BitRateBox ();      // optional 
 MPEG4ExtensionDescriptorsBox (); // optional 
} 
class AVC2SampleEntry() extends VisualSampleEntry (‘avc2’){ 
 AVCConfigurationBox  avcconfig; 
 MPEG4BitRateBox bitrate;       // optional 
 MPEG4ExtensionDescriptorsBox descr; // optional 
 extra_boxes    boxes;    // optional 
} 
```

In [18]:
class AudioSampleEntry(Box):
    def __init__(self):
        self.body_fmt = Struct(
            "reserved" / Bytes(6),
            "data_reference_index" / Int16ub,
            "reserved1" / Bytes(8),
            "channelcount" / Int16ub,
            "samplesize" / Int16ub,
            "pre_defined" / Int16ub,
            "perserved2" / Bytes(2),
            "samplerate" / Aligned(4, Int16ub),
        )

        super().__init__("mp4a")


class HintSampleEntry(Box):
    def __init__(self):
        self.body_fmt = Struct(
            "reserved" / Bytes(6),
            "data_reference_index" / Int16ub,
            "data" / Int8ub,
        )

        super().__init__()


class AvcCBox(FullBox):
    def __init__(self):
        self.body_fmt = Struct(

        )

class VisualSampleEntry(Box):
    def __init__(self):
        self.body_fmt = Struct(
            "reserved" / Bytes(6),
            "data_reference_index" / Int16ub,
            "pre_defined1" / Bytes(2),
            "reserved1" / Bytes(2),
            "pre_defined2" / Bytes(12),
            "width" / Int16ub,
            "height" / Int16ub,
            "horizresolution" / Aligned(4, Int16ub),
            "vertresolution" / Aligned(4, Int16ub),
            "reserved2" / Bytes(4),
            "frame_count" / Int16ub,
            "compressorname" / PaddedString(32, 'utf8'),
            "depth" / Int16ub,
            "pre_defined3" / Bytes(2),
        )

        super().__init__("avc1")


class SampleDescriptionBox(FullBox):
    def __init__(self, handler_type):

        if handler_type == "vide":
            self.entry_fmt = VisualSampleEntry().fmt
        elif handler_type == "soun":
            self.entry_fmt = AudioSampleEntry().fmt
        elif handler_type == "hint":
            self.entry_fmt = HintSampleEntry().fmt

        self.body_fmt = Struct(
            "entry_count" / Int32ub, "entries" / Array(this.entry_count, self.entry_fmt)
        )

        super().__init__("stsd")


vide_stsd_offset = data.find(b"stsd") - 4
if vide_stsd_offset >= 0:
    vide_stsd = SampleDescriptionBox("vide")
    vide_stsd.init(data[vide_stsd_offset:])
    print(vide_stsd)

soun_stsd_offset = data.find(b"stsd", vide_stsd_offset + 5) - 4
if soun_stsd_offset >= 0:
    soun_stsd = SampleDescriptionBox("soun")
    soun_stsd.init(data[soun_stsd_offset:])
    print(soun_stsd)


###### stco(Chunk Offset Box)

chunk在文件中的偏移量。针对小文件、大文件，有两种不同的box类型，分别是stco、co64，它们的结构是一样的，只是字段长度不同。

chunk_offset 指的是在文件本身中的 offset，而不是某个box内部的偏移。

在构建mp4文件的时候，需要特别注意 moov 所处的位置，它对于chunk_offset 的值是有影响的。有一些MP4文件的 moov 在文件末尾，为了优化首帧速度，需要将 moov 移到文件前面，此时，需要对 chunk_offset 进行改写。

定义如下:
```java
// Box Type: ‘stco’, ‘co64’
// Container: Sample Table Box (‘stbl’) Mandatory: Yes
// Quantity: Exactly one variant must be present

aligned(8) class ChunkOffsetBox
	extends FullBox(‘stco’, version = 0, 0) { 
	unsigned int(32) entry_count;
	for (i=1; i u entry_count; i++) {
		unsigned int(32)  chunk_offset;
	}
}

aligned(8) class ChunkLargeOffsetBox
	extends FullBox(‘co64’, version = 0, 0) { 
	unsigned int(32) entry_count;
	for (i=1; i u entry_count; i++) {
		unsigned int(64)  chunk_offset;
	}
}
```

In [19]:
class ChunkOffsetBox(FullBox):
    def __init__(self):
        self.body_fmt = Struct(
            "entry_count" / Int32ub,
            "chunk_offset_list" / Hex(Int32ub)[this.entry_count],
        )

        super().__init__('stco')

class ChunkLargeOffsetBox(FullBox):
    def __init__(self):
        self.body_fmt = Struct(
            "entry_count" / Int32ub,
            "chunk_offset_list" / Hex(Int64ub)[this.entry_count],
        )

        super().__init__('co64')


stco_offset = data.find(b'stco') - 4
if stco_offset >= 0:
    stco = ChunkOffsetBox()
    stco.init(data[stco_offset:])
    print(stco)


###### stsc(Sample To Chunk Box)

sample 以 chunk 为单位分成多个组。chunk的size可以是不同的，chunk里面的sample的size也可以是不同的。

- entry_count：有多少个表项（每个表项，包含first_chunk、samples_per_chunk、sample_description_index信息）；
- first_chunk：当前表项中，对应的第一个chunk的序号；
- samples_per_chunk：每个chunk包含的sample数；
- sample_description_index：指向 stsd 中 [sample description 的索引值](#stsdsample-description-box)

这里看个例子，这里表示的是：

- 序号1~15的chunk，每个chunk包含15个sample；
- 序号16的chunk，包含30个sample；
- 序号17以及之后的chunk，每个chunk包含28个sample；
- 以上所有chunk中的sample，对应的sample description的索引都是1；

| first_chunk | samples_per_chunk | sample_description_index |
| ----------- | ----------------- | ------------------------ |
| 1	 |15 | 1 |
| 16 |	30 | 1 |
| 17 |	28 | 1 |

定义如下:
```java
aligned(8) class SampleToChunkBox
	extends FullBox(‘stsc’, version = 0, 0) { 
	unsigned int(32) entry_count;
	for (i=1; i u entry_count; i++) {
		unsigned int(32) first_chunk;
		unsigned int(32) samples_per_chunk; 
		unsigned int(32) sample_description_index;
	}
}
```

In [20]:
class SampleToChunkBox(FullBox):
    def __init__(self):
        self.body_fmt = Struct(
            "entry_count" / Int32ub,
            "chunk_info_list"
            / Array(
                this.entry_count,
                Struct(
                    "first_chunk" / Int32ub,
                    "samples_per_chunk" / Int32ub,
                    "sample_description_index" / Int32ub,
                ),
            ),
        )

        super().__init__("stsc")

stsc_offset = data.find(b'stsc') - 4
if stsc_offset >= 0:
    stsc = SampleToChunkBox()
    stsc.init(data[stsc_offset:])
    print(stsc)


###### stsz（Sample Size Boxes）

每个sample的大小（字节），根据 sample_size 字段，可以知道当前track包含了多少个sample（或帧）。

有两种不同的box类型，stsz、stz2。

stsz：

- sample_size：默认的sample大小（单位是byte），通常为0。如果sample_size不为0，那么，所有的sample都是同样的大小。如果sample_size为0，那么，sample的大小可能不一样。
- sample_count：当前track里面的sample数目。如果 sample_size==0，那么，sample_count 等于下面entry的条目；
- entry_size：单个sample的大小（如果sample_size==0的话）；

```java
aligned(8) class SampleSizeBox extends FullBox(‘stsz’, version = 0, 0) { 
	unsigned int(32) sample_size;
	unsigned int(32) sample_count;
	if (sample_size==0) {
		for (i=1; i u sample_count; i++) {
			unsigned int(32)  entry_size;
		}
	}
}
```

stz2：

- field_size：entry表中，每个entry_size占据的位数（bit），可选的值为4、8、16。4比较特殊，当field_size等于4时，一个字节上包含两个entry，高4位为entry[i]，低4位为entry[i+1]；
- sample_count：等于下面entry的条目；
- entry_size：sample的大小。

```java
aligned(8) class CompactSampleSizeBox extends FullBox(‘stz2’, version = 0, 0) { 
	unsigned int(24) reserved = 0;
	unisgned int(8) field_size;
	unsigned int(32) sample_count;
	for (i=1; i u sample_count; i++) {
		unsigned int(field_size) entry_size;
	}
}
```

In [21]:
class SampleSizeBox(FullBox):
    def __init__(self):
        self.body_fmt = Struct(
            "sample_size" / Int32ub,
            "sample_count" / Int32ub,
            "sample_size_list" / Int32ub[this.sample_count],
        )

        super().__init__('stsz')


stsz_offset = data.find(b'stsz') - 4
if stsz_offset >= 0:
    stsz = SampleSizeBox()
    stsz.init(data[stsz_offset:])
    print(stsz)

class CompactSampleSizeBox(FullBox):
    def __init__(self):
        self.body_fmt = Struct(
            "reserved" / Int24ub,
            "field_size" / Int8ub,
            "sample_count" / Int32ub,
            "sample_size_list" / BitsInteger(this.field_size)[this.sample_count],
        )

        super().__init__('stz2')


stz2_offset = data.find(b'stz2') - 4
if stz2_offset >= 0:
    stz2 = CompactSampleSizeBox()
    stz2.init(data[stz2_offset:])
    print(stz2)

###### stts（Decoding Time to Sample Box）

stts包含了 [DTS](#dtsdecoding-time-stamp-和-ptspresentation-time-stamp) 到sample number的映射表，主要用来推导每个帧的时长。

- entry_count：stts 中包含的entry条目数；
- sample_count：单个entry中，具有相同时长（duration 或 sample_delta）的连续sample的个数。
- sample_delta：sample的时长（以timescale为计量）

还是看例子，如下图，entry_count为3，
前250个sample的时长为1000，
第251个sample时长为999，
第252~283个sample的时长为1000。

>假设timescale为1000，则实际时长需要除以1000。

![图 1](../asset/ef97f55fd2fe04a68c5bafbc7912fac03c87f9ce143a914a572a9afb7fddd814.png)  


```java
aligned(8) class TimeToSampleBox extends FullBox(’stts’, version = 0, 0) {
	unsigned int(32)  entry_count;
	int i;
	for (i=0; i < entry_count; i++) {
		unsigned int(32)  sample_count;
		unsigned int(32)  sample_delta;
	}
}
```

In [22]:
class TimeToSampleBox(FullBox):
    def __init__(self):
        self.body_fmt = Struct(
            "entry_count" / Int32ub,
            "entry_list"
            / Array(
                this.entry_count,
                Struct(
                    "sample_count" / Int32ub,
                    "sample_delta" / Int32ub,
                ),
            ),
        )

        super().__init__("stts")


stts_offset = data.find(b"stts") - 4
if stts_offset >= 0:
    stts = TimeToSampleBox()
    stts.init(data[stts_offset:])
    print(stts)


###### stss（Sync Sample Box）

mp4文件中，关键帧所在的sample序号。如果没有stss的话，所有的sample中都是关键帧。

- entry_count：entry的条目数，可以认为是关键帧的数目；
- sample_number：关键帧对应的sample的序号；（从1开始计算）

例子如下，第1、31、61、91、121...271个sample是关键帧。

![图 2](../asset/dde831b26f0efe11a5f445b2712133855e4f4c3aa6b953827ad6a642c36c2311.png)  

```java
aligned(8) class SyncSampleBox
   extends FullBox(‘stss’, version = 0, 0) {
   unsigned int(32)  entry_count;
   int i;
   for (i=0; i < entry_count; i++) {
      unsigned int(32)  sample_number;
   }
}
```


In [24]:
class SyncSampleBox(FullBox):
    def __init__(self):
        self.body_fmt = Struct(
            "entry_count" / Int32ub,
            "sample_number" / Int32ub[this.entry_count],
        )

        super().__init__('stss')


stss_offset = data.find(b'stss') - 4
if stss_offset >= 0:
    stss = SyncSampleBox()
    stss.init(data[stss_offset:])
    print(stss)

###### ctts（Composition Time to Sample Box）

从解码（[DTS](#dtsdecoding-time-stamp-和-ptspresentation-time-stamp)）到渲染（[PTS](#dtsdecoding-time-stamp-和-ptspresentation-time-stamp)）之间的差值。

对于只有I帧、P帧的视频来说，解码顺序、渲染顺序是一致的，此时，ctts没必要存在。

对于存在B帧的视频来说，ctts就需要存在了。
当PTS、DTS不相等时，就需要ctts了，公式为 显示时间 CT(n) = 解码时间 DT(n) + CTTS(n) 。

即CTTS(n) 查询到第n个sample的delta值（基于它解码时间的），
再加上 DT(n) 该sample（第n个）的解码时间，就是它的显示时间。
注意 这个delta可正 可负。

```java
aligned(8) class CompositionOffsetBox extends FullBox(‘ctts’, version = 0, 0) { unsigned int(32) entry_count;
      int i;
   for (i=0; i < entry_count; i++) {
      unsigned int(32)  sample_count;
      unsigned int(32)  sample_offset;
   }
}
```

In [25]:
class CompositionOffsetBox(FullBox):
    def __init__(self):
        self.body_fmt = Struct(
            "entry_count" / Int32ub,
            "entry_list" / Array(this.entry_count, Struct(
                "sample_count" / Int32ub,
                "sample_offset" / Int32ub,
            ))
        )

        super().__init__('ctts')


ctts_offset = data.find(b'ctts') - 4
if ctts_offset >= 0:
    ctts = CompositionOffsetBox()
    ctts.init(data[ctts_offset:])
    print(ctts)

# 如何计算某一帧的 Sample 偏移位置

步骤如下: 
1. 将 PTS 转换到媒体对应的时间坐标系。
2. 根据 stts 计算某个 PTS 对应的 Sample 序号。
3. 根据 stsc 计算 Sample 序号存放在哪个 Chunk 中。
4. 根据 stco 获取对应 Chunk 在文件中的偏移位置。
5. 根据 stsz 获取 Sample 在 Chunk 内的偏移位置并加上第 4 步获取的偏移，计算出 Sample 在文件中的偏移。

例如下面代码计算 3.56s 的 Sample 偏移位置

In [ ]:
pts = 7.56
print(f"pts: {pts}")
# 1. 通过 mdhd 中的 timescale 将 pts 转换成文件中的时间坐标系
pts_in_media = pts * mdhd.body.timescale
print(f"pts_in_media: {pts_in_media}")

# 2. 根据 stts 计算某个 PTS 对应的 Sample 序号。
import math


def getSampleIdxByPts(pts_in_media):
    cur_delta = 0
    cur_sample = 0
    for sample_delta in stts.body.entry_list:
        cur_sample += sample_delta.sample_count
        cur_delta += sample_delta.sample_count * sample_delta.sample_delta
        # print(sample_delta)
        if cur_delta >= pts_in_media:
            ret = math.floor(
                cur_sample - (cur_delta - pts_in_media) / sample_delta.sample_delta
            )
            return ret


sample_idx = getSampleIdxByPts(pts_in_media)
print(f"sample_idx: {sample_idx}")

# 3. 根据 stsc 计算 Sample 序号存放在哪个 Chunk 中。
def getChunkAndOffset(sample_idx):
    total_sample = 0
    for i in range(len(stsc.body.chunk_info_list)):
        cur_chunk_block = stsc.body.chunk_info_list[i]

        if i != len(stsc.body.chunk_info_list) - 1:
            next_chunk_block = stsc.body.chunk_info_list[i + 1]

            cur_chunk_block_sample_count = (
                next_chunk_block.first_chunk - cur_chunk_block.first_chunk
            ) * cur_chunk_block.samples_per_chunk

            if sample_idx > total_sample + cur_chunk_block_sample_count:
                total_sample += cur_chunk_block_sample_count
                continue

        sample_idx_in_chunk_block = sample_idx - total_sample
        chunk_idx = cur_chunk_block.first_chunk + (
            math.ceil(sample_idx_in_chunk_block / cur_chunk_block.samples_per_chunk) - 1
        )
        sample_idx_in_chunk = sample_idx_in_chunk_block - (chunk_idx - cur_chunk_block.first_chunk) * cur_chunk_block.samples_per_chunk

        return chunk_idx, sample_idx_in_chunk, sample_idx - sample_idx_in_chunk + 1


chunk_idx, sample_idx_in_chunk, chunk_first_sample_idx = getChunkAndOffset(sample_idx)

print(f"chunk_idx: {chunk_idx}")
print(f"sample_idx_in_chunk: {sample_idx_in_chunk}")
print(f"chunk_first_sample_idx: {chunk_first_sample_idx}")

# 4. 根据 stco 获取对应 Chunk 在文件中的偏移位置。
chunk_offset = stco.body.chunk_offset_list[chunk_idx - 1]
print(f"chunk_offset: {chunk_offset}")

# 5. 根据 stsz 获取 Sample 在 Chunk 内的偏移位置并加上第 4 步获取的偏移，计算出 Sample 在文件中的偏移。
sample_size = stsz.body.sample_size_list[sample_idx - 1]
sample_offset_in_chunk = 0
for i in range(chunk_first_sample_idx - 1, sample_idx - 1):
    sample_offset_in_chunk += stsz.body.sample_size_list[i]
print(f"sample_offset_in_chunk: {hex(sample_offset_in_chunk)}")
print(f"sample_size: {sample_size}")

sample_offset_in_file = chunk_offset + sample_offset_in_chunk
print(f"sample_offset_in_file: {hex(sample_offset_in_file)}")

# for sample_info in ctts.body.entry_list:


# print(ctts)


pts: 7.56
pts_in_media: 120960.0
sample_idx: 189
chunk_idx: 188
sample_idx_in_chunk: 1
chunk_first_sample_idx: 189
chunk_offset: 0x0003C520
sample_offset_in_chunk: 0x0
sample_size: 32
sample_offset_in_file: 0x3c520


# mdat(Media Data Box)

该box是实际存储数据的地方, 主要靠stbl中的信息来读取数据

定义如下:
```java
aligned(8) class MediaDataBox extends Box(‘mdat’) { 
 bit(8) data[]; 
} 
```


In [26]:
class MediaDataBox(Box):
    def __init__(self):
        self.body_fmt = Struct("data" / GreedyBytes)

        super().__init__('mdat')


mdat_offset = data.find(b'mdat') - 4
if mdat_offset >= 0:
    mdat = MediaDataBox()
    mdat.init(data[mdat_offset:])
    print(mdat)

box_name: mdat
Normal Box Container: 
    size = 1
    type = u'mdat' (total 4)
    large_size = None
    extended_type = None
    body = Container: 
        data = b'\x00\x00\x00\x00\x00\xe3\xb5\x9f!{U\x05\xc2\x8fb\x82'... (truncated, total 5157552)


## H.264比特流

H.264 的功能分为两层，即视频编码层（VCL）和网络提取层（NAL)

- VCL（Video Coding Layer）视频编码层，包括核心压缩引擎和块、宏块和片的语法级别定义，设计目标是尽可能地独立于网络进行高效的编码，负责有效表示视频数据的内容。

- NAL（Network Abstraction Layer）网络提取层，负责将 VCL 产生的比特字符串适配到各种各样的网络和多元环境中，覆盖了所有片级以上的语法级别；一个NALU 单元常由 [NALU Header] + [NALU Payload] 部分组成，NAL对VCL进行了封装包裹

### 编码方式

| 语法元素描述符	| 编码方法 |
| --- | --- |
| b(8)	| 8位二进制比特位串，用于描述rbsp_byte() |
| f(n)	| n位固定模式比特位串，从最左bit开始计算 |
| u(n)	| 使用n位无符号整数表示，由n位bit换算得到 |
| i(n)	| 使用n位有符号整数表示，由n位bit换算得到 |
| ue(v)	| 使用无符号指数哥伦布编码 |
| se(v)	| 使用有符号指数哥伦布编码 |
| te(v)	| 使用截断指数哥伦布编码 |
| me(v)	| 使用映射指数哥伦布编码 |
| ce(v)	| 上下文自适应的变长编码 |
| ae(v)	| 上下文自适应的二进制算术编码 |

特殊符号 | 符号: (例如 u(1)|ae(v)), 根据 entropy_coding_mode_flag 来选择
- entropy_coding_mode_flag = 0: 选择左边(u(1))
- entropy_coding_mode_flag = 1: 选择右边(ae(v))


#### 指数哥伦布 (Exponential-Golomb) 熵编码

[指数哥伦布熵编码](https://www.zzsin.com/article/golomb.html)

指数哥伦布编码是变长编码的，一个值是可以随着他的值的不同而有不同的容量的。这也是熵编码的主要特征之一

在 H.264 中，指数哥伦布编码又分成了 4 种：

- 无符号指数哥伦布熵编码 ue(v)
- 有符号指数哥伦布熵编码 se(v)
- 映射指数哥伦布熵编码 me(v)
- 截断指数哥伦布熵编码 te(v)

##### 无符号指数哥伦布熵编码 ue(v)

编码过程:
- 先把要编码的数字加 1，假设我们要编码的数字是 4，那么我们先对 4 加 1，就是 5。
- 将加 1 后的数字 5 先转换成二进制，就是： 101。
- 转化成二进制之后，我们看转化成的二进制有多少位，然后在前面补位数减一个 0 。例如，101 有 3 位，那么我们应该在前面补两个 0。

最后，4 进行无符号指数哥伦布编码之后得到的二进制码流就是 0 0 1 0 1。

其中第三步补 0 操作, 是为了防止两组二进制数据组合后混淆
如编码 1 0 1 和编码 1 1 0, 组合后 1 0 1 1 1 0
这样就无法解析组合后的编码了

##### 有符号指数哥伦布熵编码 se(v)

- 先把要编码的数字取绝对值后转换成二进制，例如我们要编码的数字是 -5，取绝对值就是 5 ，转换成二进制就是 1 0 1。
- 在二进制序列后增加一位符号位：0表示正，1表示负。二进制序列就变成了 1 0 1 1。
- 查看现在的二进制序列的长度，在前面补长度减一个零。二进制序列就变成了 0 0 0 1 0 1 1。

截断指数哥伦布熵编码要解决的问题其实和映射指数哥伦布熵编码要解决的问题差不多。

当语法元素以截断指数哥伦布解码时，首先需要判断的是语法元素的取值范围，假定为[0, x], x≥1。根据x的取值情况，语法元素根据不同情况进行解析。若 x>1，解析方法同无符号指数哥伦布熵编码相同。若 x=1 ，语法元素值等同于下一位bit值的取反。

##### 截断指数哥伦布熵编码 te(v)

截断指数哥伦布熵编码要解决的问题其实和映射指数哥伦布熵编码要解决的问题差不多。

当语法元素以截断指数哥伦布解码时，首先需要判断的是语法元素的取值范围，假定为[0, x], x≥1。根据x的取值情况，语法元素根据不同情况进行解析。若 x>1，解析方法同无符号指数哥伦布熵编码相同。若 x=1 ，语法元素值等同于下一位bit值的取反。

In [ ]:
import struct
from bitstream import BitStream

class ExpoGolombEntropy:
    def __init__(self, data:bytes, bit_len):
        self.data = data
        self.stream = BitStream(data)
        self.bit_len = bit_len

        self._readed_bit = 0

    def _read_bit(self):
        if self._readed_bit >= self.bit_len:
            return None
        
        self._readed_bit += 1
        return int(self.stream.read(bool))

    def _read_nbit(self, n):
        ret = 0
        while n > 0: 
            ret = ret << 1
            ret |= self._read_bit()
            n -= 1
        return ret 

    def read_ue(self):
        n = 0
        while True:
            bit = self._read_bit()
            if bit == None:
               return None 

            if bit == 0:
                n += 1
            else:
                break
        
        return self._read_nbit(n) + (1 << n) - 1

    def read_se(self):
        ue = self.read_ue()
        if ue == None:
            return None

        ue += 1
        if ue & 1:
            return -(ue >> 1)
        else:
            return ue >> 1


# ue [4, 5] 编码后为 0 0 1 0 1 0 0 1 1 0, 后续无效数据补零
golomb_ue_data = struct.pack('BB', 0b00101001, 0b10000000)
golomb = ExpoGolombEntropy(golomb_ue_data, 10)
print(golomb.read_ue())
print(golomb.read_ue())

# se [-5, 4]数据为 0 0 0 1 0 1 1 0 0 0 1 0 0 0, 后续无效数据补零
golomb_se_data = struct.pack('BB', 0b00010110, 0b00100000)
se = ExpoGolombEntropy(golomb_se_data, 14)
print(se.read_se())
print(se.read_se())


4
5
-5
4


### NALU

NALU 是 H.264 码流 NAL 层的基本单元, 其结构就是 [NALU Header] + [NALU Payload] 两部分组成

![图 1](../asset/facdf1b12d545ba7ac8c66240dc2678932aabeae3901a299c279374021ca17ca.png)  


需要注意的是, 在如何分割多个 NALU 的方式上, 我目前已知的有三种
1. Annex-B 格式: 通过起始码 0x000001或0x00000001 进行分割
2. RTP 包流: 使用RTP协议来分割, 一个NALU存储于RTP的Payload中
3. MP4中的 avcC 编码格式: 4字节整数在开头, 标识整个NALU的长度来分割


为了分析 H.264, 通过下面命令提取视频
```
ffmpeg -i test.mp4 -vcodec copy -an test_copy.h264
```

#### NALU Header
NALU Header只占1个字节，即8位，其组成如下：

```
| forbidden_zero_bit | nal_ref_idc | nal_unit_type |
`--------------------+-------------+---------------`
|        1 bit       |    2 bit    |    5 bit      |
```

- forbidden_zero_bit：禁止位，在网络传输中发生错误时，会被置为1，告诉接收方丢掉该单元；否则为0
- nal_ref_idc: 指示当前NALU的优先级，或者说重要性，数值越大表明越重要
相关详细说明参考 [Introduction to H.264: NAL Unit](https://yumichan.net/video-processing/video-compression/introduction-to-h264-nal-unit/)
>On one hand, if it is a reference field / frame / picture, nal_ref_idc is not equal to 0. According to the Recommendation, non-zero nal_ref_idc specifies that the content of the NAL unit contains a sequence parameter set (SPS), a SPS extension, a subset SPS, a picture parameter set (PPS), a slice of a reference picture, a slice data partition of a reference picture, or a prefix NAL unit preceding a slice of a reference picture.
On the other hand, if it is a non-reference field / frame / picture, nal_ref_idc is equal to 0.

>一方面，如果它是参考字段/帧/图片，则nal_ref_idc不等于0。根据H264标准，非零nal_ref_idcs指示NALU的内容可能为：序列参数集（SPS），SPS扩展，子SPS，图片参数集（PPS），参考图片的切片，参考图片的切片数据分区，或参考图片的切片之前的前缀NALU。 另一方面，如果它是非参考场/帧/图片，则nal_ref_idc等于0。

- nal_unit_type：表示NALU的类型, 完整的类型列表参考 [Introduction to H.264: NAL Unit](https://yumichan.net/video-processing/video-compression/introduction-to-h264-nal-unit/)

| nal_unit_type	| NALU 类型	|      |
| ------------- | --------- | --- |
| 0	| 未定义	| |
| 1	| 非 IDR SLICE |	slice_layer_without_partitioning_rbsp( ) |
| 2	| 非 IDR SLICE，采用 A 类数据划分片段 |	slice_data_partition_a_layer_rbsp( ) |
| 3	| 非 IDR SLICE，采用 B 类数据划分片段 |	slice_data_partition_b_layer_rbsp( ) |
| 4	| 非 IDR SLICE，采用 C 类数据划分片段 |	slice_data_partition_c_layer_rbsp( ) |
| 5	| IDR SLICE |	slice_layer_without_partitioning_rbsp( ) |
| 6	| 补充增强信息 SEI |	sei_rbsp( ) |
| 7	| 序列参数集 SPS |	seq_parameter_set_rbsp( ) |
| 8	| 图像参数集 PPS |	pic_parameter_set_rbsp( ) |
| 9	| 分隔符 |	access_unit_delimiter_rbsp( ) |
| 10	| 序列结束符 |	end_of_seq_rbsp( ) |
| 11	| 码流结束符 |	end_of_stream_rbsp( ) |
| 12	| 填充数据 |	filler_data_rbsp( ) |
| 13	| 序列参数扩展集 |	seq_parameter_set_extension_rbsp( ) |
| 14| ~18 |	保留	 |
| 19	| 未分割的辅助编码图像的编码条带 |	slice_layer_without_partitioning_rbsp( ) |
| 20| ~23 |	保留	 |
| 24| ~31 |	未指定	 |

- SPS(nal_unit_type=7)和PPS(nal_unit_type=8)包含了初始化H264解码器所需要的信息参数，包括编码所用的profile,level,图像的宽和高，deblock滤波器等。
    - SPS，全称Sequence Paramater Set，序列参数集
    - PPS，全称Picture Paramater Set，图像参数集

- I帧（nal_unit_type=5 or 1），帧内编码帧，I帧表示关键帧，可以理解为这一帧画面的完整保留，即拥有还原成图像所需的所有数据。
    - 它是一个全帧压缩帧，将全帧图像信息进行JPEG压缩编码及传输；
    - 解码时仅用I帧的数据就可以重构完整图像
    - I帧描述了图像背景和运动主体的详情。
    - I帧不需要参考其他画面生成
    - I帧是P帧和B帧的参考帧（其质量直接影响到同组中以后各帧的质量）
    - I帧不需要考虑运动矢量
    - I帧所占数据的信息量比较大
- IDR帧(nal_unit_type=5), I帧的一种，告诉解码器，之前依赖的解码参数集合（接下来要出现的SPS\PPS等）可以被刷新了
    - IDR帧一定是I帧, I帧不一定是IDR帧
    - 在传输不稳定的流媒体中十分重要
    - 在类似MP4之类的存储格式中, 因为不用担心丢包, 所以只有开头一个IDR帧

- P帧（nal_unit_type=1），前向预测编码帧。P帧表示的是这一帧跟之前的一个关键帧（或P帧）的差别，解码时需要用之前缓存的画面叠加上本帧定义的差别，生成最终画面。（也就是差别帧，P帧没有完整画面数据，只有与前一帧的画面差别的数据）
    - P帧特点
    - P帧是I帧后面相隔1~2帧的编码帧;
    - P帧采用运动补偿的方法传送它与前面的I或P帧的差值及运动矢量(预测误差);
    - 解码时必须将I帧中的预测值与预测误差求和后才能重构完整的P帧图像;
    - P帧属于前向预测的帧间编码。它只参考前面最靠近它的I帧或P帧;
    - P帧可以是其后面P帧的参考帧,也可以是其前后的B帧的参考帧;
    - 由于P帧是参考帧,它可能造成解码错误的扩散;
    - 由于是差值传送,P帧的压缩比较高。

- B帧（nal_unit_type=1），双向预测内插编码帧。B帧是双向差别帧，也就是B帧记录的是本帧与前后帧的差别，换言之，要解码B帧，不仅要取得之前的缓存画面，还要解码之后的画面，通过前后画面的与本帧数据的叠加取得最终的画面。B帧压缩率高，但是解码时CPU会比较累。
    - B帧特点
    - B帧是由前面的I或P帧和后面的P帧来进行预测的;
    - B帧传送的是它与前面的I或P帧和后面的P帧之间的预测误差及运动矢量;
    - B帧是双向预测编码帧;
    - B帧压缩比最高,因为它只反映丙参考帧间运动主体的变化情况,预测比较准确;
    - B帧不是参考帧,不会造成解码错误的扩散。

- SEI(nal_unit_type=6)，英文全称Supplemental Enhancement Information，翻译为“补充增强信息”，提供了向视频码流中加入额外信息的方法。

- AU分隔符(nal_unit_type=9)，AU全称Access Unit，它是一个或者多个NALU的集合，代表了一个完整的帧。

#### NALU Payload

Payload 真正的定义如下

```
NALU = NALU Header + SODB // 定义1
NALU = NALU Header + RBSP // 定义2
NALU = NALU Header + EBSP // 定义3
```

SODB, RBSP, EBSP 详细的定义可以参考[这里](https://www.jianshu.com/p/5f89ea2c3a28), 下面做简单的介绍

**SODB**

英文全称String Of Data Bits，称原始数据比特流，就是最原始的编码/压缩得到的数据。

**RBSP**

全称Raw Byte Sequence Payload，又称原始字节序列载荷。和SODB关系如下：

>RBSP = SODB + RBSP Trailing Bits（RBSP尾部补齐字节）

引入RBSP Trailing Bits做8位字节补齐。

补齐的规则如下:
```java
rbsp_trailing_bits( ) {
    rbsp_stop_one_bit /* equal to 1 */
    while( !byte_aligned( ) )
        rbsp_alignment_zero_bit /* equal to 0 */
}
```

先写入 1 bit 数据，数据内容是 1，然后开始补齐 0，直到补齐到一整个字节
![图 3](../asset/128beae56c0d653c74d8d9261857acf8f884226a67e189b292538440c8918279.png)  

这里其实还有另外一种情况，那就是你的码流在写入的时候，刚刚好写满了一整个字节, 这种情况会新增一个字节
![图 4](../asset/ca1f1e0d867324a46c538d856d3b2970373c12c3b7870e104db3a3b78cb59889.png)  


**EBSP**

全称Encapsulated Byte Sequence Payload，称为扩展字节序列载荷。和RBSP关系如下：

>EBSP ：RBSP插入防竞争字节（0x03）

防止竞争字节（0x03）时为了防止 H264 Annex-B格式的StartCode（0x000001或0x00000001）出现在 RBSP 中导致歧义而添加的

- 编码时，扫描RBSP，如果遇到连续两个0x00字节，就在后面添加防止竞争字节（0x03）；
- 解码时，同样扫描EBSP，进行逆向操作即可。

![图 2](../asset/a852945985ae69a95d92c8a6e8caa3a1cb693dbf27edd9bcb0869fd95bd46a30.png)  

In [ ]:
class NALU:
    def __init__(self, data):
        self.data = data
        self.fmt = Struct(
            "Header"
            / BitStruct(
                "forbidden_zero_bit" / Flag,
                "nal_ref_idc" / BitsInteger(2),
                "nal_util_type"
                / Enum(
                    BitsInteger(5),
                    non_IDR=1,
                    coded_slice_data_partition_a=2,
                    coded_slice_data_partition_b=3,
                    coded_slice_data_partition_c=4,
                    IDR=5,
                    SEI=6,
                    SPS=7,
                    PPS=8,
                    access_unit_delimiter=9,
                    end_of_seq=10,
                    end_of_stream=11,
                    filler_data=12,
                ),
            ),
            "Payload" / GreedyBytes,
        )

        self.struct = self.fmt.parse(self.data)

    def __str__(self):
        return self.struct.__str__()


class AnnexB_NALU:
    def __init__(self, data):
        self.data = data
        self.start_code = [b"\x00\x00\x01", b"\x00\x00\x00\x01"]
        self.nalu_list = []

    def _find_start_code_pos(self, offset):
        ret_pos = -1
        start_code_len = 0
        for s in self.start_code:
            pos = self.data.find(s, offset)
            if pos == -1:
                continue
            if ret_pos == -1:
                ret_pos = pos
                start_code_len = len(s)
            elif pos < ret_pos:
                ret_pos = pos
                start_code_len = len(s)
        return ret_pos, start_code_len

    def init(self):
        begin_pos, code_len = self._find_start_code_pos(0)
        next_begin_pos = 0
        while next_begin_pos != -1:
            next_begin_pos, next_code_len = self._find_start_code_pos(
                begin_pos + code_len
            )
            if next_begin_pos == -1:
                self.nalu_list.append(NALU(self.data[begin_pos + code_len :]))
            else:
                self.nalu_list.append(
                    NALU(self.data[begin_pos + code_len : next_begin_pos])
                )
                begin_pos = next_begin_pos
                code_len = next_code_len


class Avc1_NALU:
    def __init__(self, data):
        self.data = data
        self.nalu_list = []

    def init(self):
        offset = 0
        data_len = len(self.data)
        fmt = Struct("size" / Int32ub, "data" / Bytes(this.size))

        while offset < data_len:
            try:
                unit = fmt.parse(self.data[offset:])
                offset += unit.size + 4
                nalu = NALU(unit.data)
            except:
                break
            self.nalu_list.append(nalu)


h264_data = None
with open("./data/data1.h264", "rb") as f:
    h264_data = f.read()

annexB_nalu = AnnexB_NALU(h264_data)
annexB_nalu.init()
print('-'*30, 'AnnexB_NALU', '-'*30)
for i, nalu in enumerate(annexB_nalu.nalu_list):
    if i >= 4:
        break
    print(nalu)

avc1_nalu = Avc1_NALU(mdat.body.data)
avc1_nalu.init()
print('-'*30, 'Avc1_NALU', '-'*30)
for i, nalu in enumerate(avc1_nalu.nalu_list):
    if i >= 4:
        break
    print(nalu)

------------------------------ AnnexB_NALU ------------------------------
Container: 
    Header = Container: 
        forbidden_zero_bit = False
        nal_ref_idc = 0
        nal_util_type = (enum) access_unit_delimiter 9
    Payload = b'\xf0' (total 1)
Container: 
    Header = Container: 
        forbidden_zero_bit = False
        nal_ref_idc = 3
        nal_util_type = (enum) SPS 7
    Payload = b'd\x00\x1e\xac\xcap(\x0b\xfe\\\x05\xa8\x08\x08\n\x00'... (truncated, total 27)
Container: 
    Header = Container: 
        forbidden_zero_bit = False
        nal_ref_idc = 3
        nal_util_type = (enum) PPS 8
    Payload = b'\xeb\xef,' (total 3)
Container: 
    Header = Container: 
        forbidden_zero_bit = False
        nal_ref_idc = 0
        nal_util_type = (enum) SEI 6
    Payload = b'\x05c\xd3\xec&\xe3\xac-T\xbf\x9c\xda\xc6\xf0\xa5\xe4'... (truncated, total 102)
------------------------------ Avc1_NALU ------------------------------
Container: 
    Header = Container: 
        

### 宏块

在 H.264 中，通常不会逐个像素去处理图像，而是把图像按照固定的长宽划分成一个一个小块，然后按照小块为单位去处理，而这些小块，我们就称之为宏块。

假如我们有一幅图像，宽度为 176 个像素，高度为 144 个像素。按照 H.264 的标准的规定，一个宏块宽为 16 个像素，高为 16 个像素，那么这个图像就是一个 11 * 9 的方格块

一般情况下，宏块可以分为两个部分，那就是亮度块（Luma）和色度块（Chroma）

- 亮度块(Luma): 大小固定为 16 * 16 个像素, 里面只存放了亮度, 也就是 YUV 中的 Y
- 色度块(Chroma): 色度块有两种, 存放 U 分量的 Chroma Cr, 存放 V 分量的 Chroma Cb. 他们的大小根据采样方式而有所变化
    - YUV 420: 两个色度块都是 8 * 8
    - YUV 422: 两个色度块都是 16 * 8
    - YUV 444: 两个色度块都是 16 * 16

![图 7](../asset/7eb14c9b38301570d623012dc84a7cdd55d2576bf401b1394f6a34c5268dcf71.png)  


所以, 一个 YUV 420 的宏块, 就像下面这样

![图 9](../asset/2d01b019d62fd054232db29be7523ad13eb8955f4c3a755b223bdcdf2ff4a5f5.png)  


### SPS 与 PPS

- 从码流中拿到 SPS 和 PPS 的原始数据，实际上是经过一次无损压缩的, 这个过程被称之为熵编码，熵编码是一类编码的总称，而在 SPS 和 PPS 中主要使用的是一种叫做指数哥伦布编码的熵编码算法
- H.264 码流中的操作单位是 bit，而不是 byte

#### SPS 与 PPS 的格式

下图是 [官方文档](https://www.itu.int/rec/dologin_pub.asp?lang=e&id=T-REC-H.264-201304-S!!PDF-E&type=items) 中 SPS 的定义

![图 1](../asset/a4e744816ebff775ded91ce9ffc49a4a2c29998464e00592fa52cea379ab042e.png)  

PPS 的定义

![图 1](../asset/3f7f4831d3a51ca1e63128cd591f6288eb75c356ec96648ae1651761a8588d99.png)  


实际的数据分布图如下:

![图 2](../asset/90c917debefbd530cdf511bb5602025b680b6d6a897751bc3dd04fb4fe8c66c6.png)  

##### profile_idc

SPS 中 profile_idc 表示的是当前这路码流的编码档次，我们有时候会说一路码流是 Baseline，Main 等等，就是这个属性所决定的。具体的映射表如下：

- Baseline profile 66
- Main profile 77
- Extended profile 88
- High profile 100
- High 10 profile 110
- High 4:2:2 profile 122
- High 4:4:4 Predictive profile 244
- High 10 Intra profile 100 or 110
- High 4:2:2 Intra profile 100, 110, or 122
- High 4:4:4 Intra profile 44, 100, 110, 122, or 244
- CAVLC 4:4:4 Intra profile 44

##### pic_width_in_mbs_minus1 和 pic_height_in_map_units_minus1

这里需要理解 [宏块](#宏块) 的概念

- pic_width_in_mbs_minus1 表示的值是横向宏块的个数减 1
- pic_height_in_map_units_minus1 表示的值是纵向宏块的个数减 1

也就是说，把他们两个加上 1，就可以获得横向的宏块个数和纵向的宏块个数。
一个宏块长 16 个像素，高 16 个像素，就可以得出视频宽高的计算公式了：

```
width = (pic_width_in_mbs_minus1 + 1) * 16;
height = (pic_height_in_map_units_minus1 + 1) * 16;
```

##### frame_crop_left_offset, frame_crop_right_offset, frame_crop_top_offset, frame_crop_bottom_offset

在 SPS 中，还有四个量，他们在 frame_cropping_flag 为 1 的时候显式存放，其他时候为 0
这四个值分别代表图像的上下左右的偏移

因为宏块只能表示 16 的倍数，当图像的宽高不是 16 的倍数的时候, 
H.264 会在图像的边沿添加一些像素，来补齐成 16 的倍数

然后通过 frame_crop_left_offset, frame_crop_right_offset, frame_crop_top_offset, frame_crop_bottom_offset 这 4 个量来记录上下左右补齐了多少数据, 
我们把这个过程称之为 Crop。

![图 1](../asset/f96817cbbca6eb0318c0bf243763880c06930c3b828dc58416a017909508d32d.png)  

但是并不是说用宏块数 * 16 然后 - 4 个 crop 的量就能得到图像的原始宽高, 还分几种情况的

##### 场编码

场编码是历史产物, 早期的电视并不能逐行扫描所有的像素, 通过视觉残留, 
通过隔行扫描, 只扫描一半来显示图像

- 帧编码容易理解，表示将一帧图像划分为16x16大小宏块进行编码，这种方式编码图像中所有宏块都是帧宏块
- 场编码，将一帧图像根据奇数行和偶数行划分为两部分，分别成为顶场和底场，如下图所示。这种方式编码的slice中所有宏块都是场宏块。

![图 2](../asset/1bf95084e37045cb397bdc4010b0a0dd0870dd18d5ded42e55326aa8e8cd3397.png)  


在场编码中，一帧等于两场，场的竖直宏块数量是帧的一半，
而在场中，crop 1 像素，相当于对帧 crop 2 个像素，所以 frame_crop_xxx_offset 也要做特殊处理

在 SPS 中，frame_mbs_only_flag 用来表示场编码相关信息
- frame_mbs_only_flag 等于 1 的时候，表示都是帧编码
- frame_mbs_only_flag 等于 0 的时候，表示有可能存在场编码

##### YUV 420, YUV 422, YUV 444 的 Crop

- YUV 444 的像素分布如下:
```
[ Y U V ]  [ Y U V ]  [ Y U V ]  [ Y U V ]
[ Y U V ]  [ Y U V ]  [ Y U V ]  [ Y U V ]
[ Y U V ]  [ Y U V ]  [ Y U V ]  [ Y U V ]
[ Y U V ]  [ Y U V ]  [ Y U V ]  [ Y U V ]
```
如果移除第一列的像素, 第二列的像素也能够正常解码, 所以宽度可以 crop 1 像素
如果移除第一行的像素, 第二行的像素也能够正常解码, 所以高度可以 crop 1 像素

- YUV 422 的像素分布如下:
```
[ Y U ]  [ Y V ]  [ Y U ]  [ Y V ]
[ Y V ]  [ Y U ]  [ Y V ]  [ Y U ]
[ Y U ]  [ Y V ]  [ Y U ]  [ Y V ]
[ Y V ]  [ Y U ]  [ Y V ]  [ Y U ]
```
如果移除第一列的像素, 第二列的像素则缺少了 U或V 分量, 无法解码, 所以宽度只能 crop 偶数
如果移除第一行的像素, 第二行的像素也能够正常解码, 所以高度可以 crop 1 像素

- YUV 420 的像素分布如下:
```
[ Y U ]  [ Y ]  [ Y U ]  [ Y ]
[ Y V ]  [ Y ]  [ Y V ]  [ Y ]
[ Y U ]  [ Y ]  [ Y U ]  [ Y ]
[ Y V ]  [ Y ]  [ Y V ]  [ Y ]
```
如果移除第一列的像素, 第二列的像素则缺少了 U, V 分量, 无法解码, 所以宽度只能 crop 偶数
如果移除第一行的像素, 第二行的像素则缺少了 U 分量, 无法解码, 所以高度只能 crop 偶数

##### chroma_format_idc

首先，我们得先知道我们的码流是什么格式的。在 SPS 中，有一个用来标记原始数据格式的语法元素叫做 chroma_format_idc，他的值如下表：

| 值	| 含义 |
| ----- | ---- |
| chroma_format_idc = 0	| 单色 |
| chroma_format_idc = 1	| YUV 4:2:0 |
| chroma_format_idc = 2	| YUV 4:2:2 |
| chroma_format_idc = 3	| YUV 4:4:4 |

![图 3](../asset/0c9ad3ea8486aaaebd87e0647b9e70c38beda9c4d584da8be2c7dc07cc0d5e80.png)  

查看码表可以知道，只有当 profile_idc 等于这些值的时候，chroma_format_idc 才会被显式记录, 
那么如果 profile_idc 不是这些值，chroma_format_idc 就会取默认值。

chroma_format_idc 的默认值是 1, 也就是 YUV 420。

##### separate_colour_plane_flag

当在码流中读取到 chroma_format_idc 之后，我们紧接着就会看到另外一个量：

![图 4](../asset/65261302c52cafae9d1385d81a318522865dc951362e7b664d0d3ecb7f76c4b3.png)  

这个语法元素在 chroma_format_idc 等于 3，也就是 YUV 444 模式的时候才有。

当图像是 YUV 444 的时候，YUV 三个分量的比重是相同的，那么就有两种编码方式了。

- separate_colour_plane_flag = 0, 就是和其他格式一样，让 UV 分量依附在 Y 分量上
- separate_colour_plane_flag = 1, 就是把 UV 和 Y 分开，独立出来

separate_colour_plane_flag 这个值默认是 0，表示 UV 依附于 Y，和 Y 一起编码，
如果 separate_colour_plane_flag 变成 1，则表示 UV 与 Y 分开编码。而对于分开编码的模式，我们采用和单色模式一样的规则。

##### ChromaArrayType
这个量在 H.264 标准文档中出现过很多次，却不是语法表格中的内容

这个值是由 separate_colour_plane_flag 和 chroma_format_idc 共同作用推导出来的，推导过程如下：

```c
if (separate_colour_plance_flag == 0){
    ChromaArrayType = chroma_format_idc;
}
else{
    ChromaArrayType = 0;
}
```

##### SubWidthC 和 SubHeightC

SubWidthC 和 SubHeightC 表示的是 YUV 分量中，Y 分量和 UV 分量在水平和竖直方向上的比值。
当 ChromaArrayType 等于 0 的时候，表示只有 Y 分量或者表示 YUV 444 的独立模式，
所以 SubWidthC 和 SubHeightC 没有意义。

![图 1](../asset/ea11b15e1de79404d1d1a3ec1f16a09ad6c1f371a2171d805363972d41061209.png)  

- YUV 420: 水平和垂直方向上，都是 2 个 Y 共用 1 对 UV，所以 SubWidthC 为 2，SubHeightC 为 2

- YUV 422: 水平方向上，是 2 个 Y 共用 1 对 UV，所以 SubWidthC 为 2；竖直方向上，是 1 个 Y 用 1 对 UV，所以 SubHeightC 为 1

- YUV 444: 水平方向上，是 1 个 Y 用 1 对 UV，所以 SubWidthC 为 1；竖直方向上，是 1 个 Y 用 1 对 UV，所以 SubHeightC 为 1

完整的推导代码如下：

```c
if (ChromaArrayType == 1) {
    SubWidthC = 2;
    SubHeightC = 2;
}
else if (ChromaArrayType == 2) {
    SubWidthC = 2;
    SubHeightC = 1;
}
else if (ChromaArrayType == 3) {
    SubWidthC = 1;
    SubHeightC = 1;
} 
// 当 ChromaArrayType = 0 的时候， SubWidthC 和 SubHeightC 无用
```

##### 最终计算实际宽高的公式

```c
width = (pic_width_in_mbs_minus1 + 1) * 16;
height = (2 - frame_mbs_only_flag) * (pic_height_in_map_units_minus1 + 1) * 16;

if(frame_cropping_flag){
    int crop_unit_x = 0;
    int crop_unit_y = 0;

    if(ChromaArrayType == 0){
        crop_unit_x = 1;
        crop_unit_y = 2 - frame_mbs_only_flag;
    }
    else if(ChromaArrayType == 1 || ChromaArrayType == 2 || ChromaArrayType == 3){
        crop_unit_x = SubWidthC;
        crop_unit_y = SubHeightC * (2 - frame_mbs_only_flag);
    }

    width -= crop_unit_x * (frame_crop_left_offset + frame_crop_right_offset);
    height -= crop_unit_y * (frame_crop_top_offset + frame_crop_bottom_offset);
}
```

### IDR
其实 IDR 就是一种特殊的 [Slice](#slice)，翻看标准文档，IDR 和普通 Slice 只有一个地方不一样

在判断是 IDR 的时候，需要多解析一个 idr_pic_id 的量。除此之外，IDR 的处理方式和普通 Slice 是完全一样的

![图 2](../asset/69684f07336c20304e87f9e3ea90fc81b0e5d7ff13d98596e81e90a27133e027.png)  

### Slice
SPS 和 PPS 中储存的信息是一些参数项，例如，图像的长宽，图像的 profile 信息等

那么在 Slice 中，存放的信息就是编码后的图像信息了

一个 Slice 通常被分为两个部分，Slice Header 和 Slice Body

![图 3](../asset/6881b8cef31d25a56ce4c6f6c9e36b3307e96cdd3c96552442c9acfb88ac67b7.png)  

- Slice Header: 存放了这个 Slice 会用到的参数项
- Slice Body: 存放了真正的图像信息

#### Slice Header

![图 4](../asset/0929e5512f8713499178bb2ca22f44c1299b11488c70354eaa0d642e058ed036.png)  


##### first_mb_in_slice

first_mb_in_slice，这个属性表示的是在这个 Slice 中第一个宏块的序号

在编码的时候，一帧图像首先会被分割成若干宏块，然后对每个宏块进行编码压缩，
之后，会将宏块写入到 Slice 里面，最后再加上一些修饰变成 NALU。
这个过程中有一点要注意，那就是一帧图像产生宏块，不一定要写到一个 Slice 里面

例如，我将一帧图像编码成了 99 个宏块，
我可以把这 99 个宏块都写到一个 Slice 里面，也可以把 0 - 30 的宏块写到一个 Slice 里面，
然后把 31-99 的宏块写到下一个宏块里。
因此，first_mb_in_slice 这个属性要描述的，
就是在当前这个 Slice 里面的第一个宏块，对应的是帧里的第几个宏块

![图 5](../asset/e4dc125d2a7264345e9a2de1bf48c18e4f4ec010d946c06268d225ae6e2a6427.png)  

在上图中，把一帧图像分割成序号为 0-23 共计 24 个宏块。

把蓝色背景的宏块放到一个 Slice 里面，这个 Slice 的 first_mb_in_slice 就是 0。

把红色背景的宏块放到另外一个 Slice 里面，这个 Slice 的 first_mb_in_slice 就是 14

##### slice_type

这个属性来指明的是 slice 的类型

| 值	| 含义 |
| --- | --- |
| 0	| P(P Slice) |
| 1	| B(B Slice) |
| 2	| I(I Slice) |
| 3	| SP(SP Slice) |
| 4	| SI(SI Slice) |
| 5	| P(P Slice) |
| 6	| B(B Slice) |
| 7	| I(I Slice) |
| 8	| SP(SP Slice) |
| 9	| SI(SI Slice) |

注意，这张表里的 slice_type 其实 5-9 和 0-4 是完全相同的，
因此在使用的时候，我们也常常直接用 slice_type 对 5 去余来处理

有人说根据 NALU 的类型就能判读是 I Slice 还是 P Slice 还是 B Slice。
其实是错误的，要判断这些必须要读到 slice_type 才行

##### pic_parameter_set_id

这个属性表明该 Slice 要依赖的 PPS 的 id

在解析 PPS 的时候，PPS 里面有一个 pic_parameter_set_id 的属性。
当你解析出 Slice 中的 pic_parameter_set_id 之后，拿着这个 id 找到与之对应的 PPS 就可以了

然后 PPS 里还有个 seq_parameter_set_id，用这个 id 就可以找到依赖的 SPS 的 id。
这样，你就可以为这个 Slice 查找到合适的 SPS 和 PPS 了

![图 6](../asset/86042c411861c0efd8563f29cbe74214489c51df6e7f4f55f4bf9df4cb0d93d2.png)  

### 帧内编码

帧内编码就是针对一帧（或者不满一帧）的数据，进行编码。
编码时所参考的数据就是这一帧，换言之，解码也就只需要这一帧数据。
这种编码方式也被用于图片的编码中，例如，jpeg 图片的编码方式就和 H.264 的帧内编码方式极度相似。

比如从一副图像中选出一个 4 x 4 的像素块(只有亮度)

![图 12](../asset/68928647f6635f2e323b9c60d2f6e359984a2551dfcfbe7d4c719258a35aa0ee.png)  

为了将所有数值都控制在一个相近的范围内, H.264 用了一种叫做预测的方式来对数据进行处理

通常情况下，在正常的一幅图像中，某一个像素，或者某一个区域的像素，总是和其周围的像素相近，或者是有关联的

比如该像素块的每个数值都与其上一行比较相似, 那么就可以用他的上一行来预测下面的像素值, 
其实就是将上一行的值赋值给像素块的每一行

![图 13](../asset/ec5ce2ef20d1f166a234921d01bbefac2bdd96faa7b736a1f51f8e0c2fa6b77b.png)  

这时, 用原像素减去预测像素, 就得到了一个差值, 这个差值叫做残差.
这样, 所有残差都是比较小的数值, 也就比较好压缩了

![图 14](../asset/80d25829424783cd892c02a62b061aa8d6503d4678c3119cd08e3c42b22f28fb.png)  

H.264 的帧内编码原理就是通过 "预测" + "残差" 来实现的

#### 帧内预测

在 H.264 中，亮度块的预测模式可以分为三种
- 16 x 16 的预测模式
- 4 x 4 的预测模式
- 8 x 8 的预测模式

一个宏块的亮度部分，就是一个 16 x 16 的像素块, 那么有两种预测方法
- 将整个宏块作为一个整体进行预测
- 也可以将这个宏块分割成 16 个 4 x 4 的小的子块，对这些子块分别进行预测。

H.264 为我们提供了两种不同颗粒度的方式，就是为了在细节多的地方使用较细致的预测方式来保留细节，
而在细节不那么多的区域使用较粗略的预测模式，来节省空间。

除了不同的颗粒度，H.264 还提供了很多种不同的预测模式，以适应更多的情况。
预测得越准，残差就越小。

##### 4 x 4 亮度块预测

将一个宏块分割成 16 个 4 x 4 的子块

![图 11](../asset/73b79616a39d989c00593c4d056f44776c9870a6c19efe6f51e03421dacebdfd.png)  

###### 4 x 4 亮度块的分割方式

对于一个宏块来说，他的 16 个子块的解码顺序并不是依次排列的，而是依照以下的划分方式。

先将宏块划分成 4 个 8 x 8 的块，然后再将每个 8 x 8 的块划分成 4 个 4 x 4 的块

![图 10](../asset/a247883abab538d4295559c54e623218a698fae3b7e983d1eec034348def6359.png)  

整个宏块中 16 个 4 x 4 的子块的解码顺序如下图:

![图 15](../asset/e5bae13525d6b1e45c143228421b7972380d80c629b7d1182ee2afdf4f04452a.png)  


###### 4 x 4 亮度块预测模式

对于 4 x 4 的子块，有 9 种预测模式

对于 4 x 4 的子块，可以参考的像素是子块上方和左边的共 13 个像素。就是下图中蓝色的色块

![图 17](../asset/9aaf2942c519f8f1517af899f1725d65a14091228f02acec8f6c998b5bba4b2e.png)  

- 竖直预测模式 vertical

![图 18](../asset/7f8d8f030fd0b3665f1e7edbc2b044eab2cb5e3150f8acba5f803377a5aad97e.png)  

- 水平预测模式 horizontal

![图 19](../asset/6ddcc7ffce6dfcf8e3fac78deb83b5a423b75f2ee51b7db14dc4fe8f5b5761bd.png)  

- DC 模式

DC 模式，又被称之为均值模式。
均值模式下，4 x 4 子块中 16 个像素都是相同的值，是上方的四个像素 A B C D 和左边四个像素 I J K L 的均值。
就是下图中红框框

![图 20](../asset/83bd335df19ec0444f2141621271c2da4fbf140d9b195e4125d9afafb4309649.png)  

而细分之下，又可以分成 4 种情况：

1. 上方 4 个像素和左边 4 个像素都不存在，例如，画面最左上角的那个 4 x 4 的块。这种情况下，所有的像素就等于：

```c
1 << (8 - 1)
```
也就是 128

2. 上方 4 个像素存在，左边 4 个像素不存在：

```c
(A + B + C + D + 2) >> 2
```

3. 上方 4 个像素不存在，左边 4 个像素存在：

```c
(I + J + K + L + 2) >> 2
```

4. 上方 4 个像素和左边 4 个像素都存在：

```c
(A + B + C + D + I + J + K + L + 4) >> 3
```

- diagnal down-left 模式

![图 21](../asset/18608a820cf66d5d0efadf3dd07caf87c06e235b4b4d800a342bde61d4caf2b6.png)  

- diagnal down-right 模式

![图 22](../asset/6f8422f3c07f92735ed2faf81ef2a733c82b30b8ff16f286578ca98cd99b3f81.png)  

- vertical-right 模式

![图 23](../asset/a106bf7ef1e5b61e2715c08e94d39c1e1cf76b79a673184b71b3aca85b303a49.png)  

- horizontal-down 模式

![图 24](../asset/310574ac87ae075f4b5616f682254508368e8242e1ee8d0b9671994cc307f105.png)  

- vertical-left 模式

![图 25](../asset/751eff977dec2ef56243df6f073c976b1c08383ab8ab3b9c11cbb327185be962.png)  

- horizontal-up 模式

![图 16](../asset/63d1be77c044195318db7469205ba948c534efe05ba0103c1e54f16f6ce07886.png)  

### 临时记录一些解码中的函数

#### 计算宏块在图像中的位置(Inverse macroblock scanning process)

下面所有计算只针对宏块中的 Y 分量

- 输入: mbAddr 宏块在宏块序列中的索引
- 输出: (x, y) 宏块左上角宏块左上角第一个像素, 相对于图片左上角第一个像素的坐标值

图中每个方块内的 1, 2, 3, ... 就是 mbAddr 的值. 1(0,0) 就代表该 mbAddr 对应的 (x, y) 为 (0, 0)

**帧编码**:

帧编码中每个宏块大小都为 16 x 16, 所以输出的 x, y 都是 16 的倍数

**帧场自适应编码**:

帧场自适应编码情况有点复杂, 他的基本单位是一个一个的宏块对(macroblock pair), 
有两种类型的宏块对: 
- 帧宏块对(frame macroblock pair): 其中包含两个帧宏块, 以中间为分界线, mbAddr 增长顺序从上到下递增
- 场宏块对(field macroblock pair): 其中包含两个场宏块, 按1像素隔行分割, 偶数行为顶场, 奇数行为底场
    - 顶场的 x, y 就是宏块对的左上角
    - 底场的 x, y 则是宏块对左上角下面一个像素


![图 2](../asset/a636832840df74b96d33e285cb52931849b7570091fd78b2b0d2e90d862cec86.png)  

计算的代码如下:
```python
def InverseRasterScan(mb_idx, mb_width, mb_height, pic_width, x_or_y):
    if x_of_y == 0:
        return (mb_idx % (pic_width // mb_width) * mb_width)
    else:
        return (mb_idx % (pic_width // mb_width) * mb_height)

if MbaffFrameFlag == 0:
    x = InverseRasterScan(mbAddr, 16, 16, PicWidthInSamples_L, 0)
    y = InverseRasterScan(mbAddr, 16, 16, PicWidthInSamples_L, 1)
elif MbaffFrameFlag == 1:
    # mbAddr // 2 是为了获取宏块对的索引
    x = InverseRasterScan(mbAddr // 2, 16, 32, PicWidthInSamples_L, 0)
    y = InverseRasterScan(mbAddr // 2, 16, 32, PicWidthInSamples_L, 1)
    # 如果该宏块为帧宏块
    if current_macroblock_is_frame_macroblock:
        # mbAddr % 2 是为了获取宏块在宏块对中的索引
        y += (mbAddr % 2) * 16
    # 如果该宏块为场宏块
    elif current_macroblock_is_field_macroblock 
        y += mbAddr % 2
```